In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_ap

In [2]:
%time hist1 = train_model(env, model, total_episodes=5500)

C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -19.000000. running mean: -19.000000
episode 2.000000, reward total was -19.000000. running mean: -19.000000
episode 3.000000, reward total was -20.000000. running mean: -19.010000
episode 4.000000, reward total was -20.000000. running mean: -19.019900
episode 5.000000, reward total was -21.000000. running mean: -19.039701
episode 6.000000, reward total was -18.000000. running mean: -19.029304
episode 7.000000, reward total was -21.000000. running mean: -19.049011
episode 8.000000, reward total was -20.000000. running mean: -19.058521
episode 9.000000, reward total was -18.000000. running mean: -19.047936
episode 10.000000, reward total was -21.000000. running mean: -19.067456
episode 11.000000, reward total was -21.000000. running mean: -19.086782
episode 12.000000, reward total was -21.000000. running mean: -19.105914
episode 13.000000, reward total was -20.000000. running mean: -19.114855
episode 14.000000, reward total was -18.000000. running mean

episode 114.000000, reward total was -20.000000. running mean: -19.946524
episode 115.000000, reward total was -21.000000. running mean: -19.957059
episode 116.000000, reward total was -21.000000. running mean: -19.967488
episode 117.000000, reward total was -20.000000. running mean: -19.967813
episode 118.000000, reward total was -20.000000. running mean: -19.968135
episode 119.000000, reward total was -21.000000. running mean: -19.978454
episode 120.000000, reward total was -20.000000. running mean: -19.978669
episode 121.000000, reward total was -21.000000. running mean: -19.988883
episode 122.000000, reward total was -20.000000. running mean: -19.988994
episode 123.000000, reward total was -17.000000. running mean: -19.959104
episode 124.000000, reward total was -21.000000. running mean: -19.969513
episode 125.000000, reward total was -21.000000. running mean: -19.979818
episode 126.000000, reward total was -19.000000. running mean: -19.970019
episode 127.000000, reward total was -

episode 225.000000, reward total was -20.000000. running mean: -20.128229
episode 226.000000, reward total was -21.000000. running mean: -20.136947
episode 227.000000, reward total was -20.000000. running mean: -20.135577
episode 228.000000, reward total was -21.000000. running mean: -20.144221
episode 229.000000, reward total was -21.000000. running mean: -20.152779
episode 230.000000, reward total was -19.000000. running mean: -20.141251
episode 231.000000, reward total was -21.000000. running mean: -20.149839
episode 232.000000, reward total was -20.000000. running mean: -20.148340
episode 233.000000, reward total was -18.000000. running mean: -20.126857
episode 234.000000, reward total was -19.000000. running mean: -20.115588
episode 235.000000, reward total was -21.000000. running mean: -20.124433
episode 236.000000, reward total was -21.000000. running mean: -20.133188
episode 237.000000, reward total was -20.000000. running mean: -20.131856
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -19.927204
episode 337.000000, reward total was -20.000000. running mean: -19.927932
episode 338.000000, reward total was -19.000000. running mean: -19.918653
episode 339.000000, reward total was -19.000000. running mean: -19.909467
episode 340.000000, reward total was -19.000000. running mean: -19.900372
episode 341.000000, reward total was -20.000000. running mean: -19.901368
episode 342.000000, reward total was -21.000000. running mean: -19.912355
episode 343.000000, reward total was -18.000000. running mean: -19.893231
episode 344.000000, reward total was -19.000000. running mean: -19.884299
episode 345.000000, reward total was -20.000000. running mean: -19.885456
episode 346.000000, reward total was -19.000000. running mean: -19.876601
episode 347.000000, reward total was -18.000000. running mean: -19.857835
episode 348.000000, reward total was -20.000000. running mean: -19.859257
episode 349.000000, reward total was -

...

episode 5019.000000, reward total was -3.000000. running mean: -3.368094
episode 5020.000000, reward total was -3.000000. running mean: -3.364413
episode 5021.000000, reward total was -6.000000. running mean: -3.390769
episode 5022.000000, reward total was 6.000000. running mean: -3.296861
episode 5023.000000, reward total was 4.000000. running mean: -3.223893
episode 5024.000000, reward total was 7.000000. running mean: -3.121654
episode 5025.000000, reward total was 9.000000. running mean: -3.000437
episode 5026.000000, reward total was 3.000000. running mean: -2.940433
episode 5027.000000, reward total was 8.000000. running mean: -2.831028
episode 5028.000000, reward total was 9.000000. running mean: -2.712718
episode 5029.000000, reward total was -13.000000. running mean: -2.815591
episode 5030.000000, reward total was 1.000000. running mean: -2.777435
episode 5031.000000, reward total was -5.000000. running mean: -2.799661
episode 5032.000000, reward total was 10.000000. running m

episode 5132.000000, reward total was 4.000000. running mean: -3.062660
episode 5133.000000, reward total was 3.000000. running mean: -3.002033
episode 5134.000000, reward total was -4.000000. running mean: -3.012013
episode 5135.000000, reward total was -6.000000. running mean: -3.041893
episode 5136.000000, reward total was -4.000000. running mean: -3.051474
episode 5137.000000, reward total was -9.000000. running mean: -3.110959
episode 5138.000000, reward total was -5.000000. running mean: -3.129850
episode 5139.000000, reward total was -4.000000. running mean: -3.138551
episode 5140.000000, reward total was 2.000000. running mean: -3.087166
episode 5141.000000, reward total was 3.000000. running mean: -3.026294
episode 5142.000000, reward total was -12.000000. running mean: -3.116031
episode 5143.000000, reward total was -8.000000. running mean: -3.164871
episode 5144.000000, reward total was 5.000000. running mean: -3.083222
episode 5145.000000, reward total was -3.000000. runnin

episode 5245.000000, reward total was -5.000000. running mean: -2.887130
episode 5246.000000, reward total was -1.000000. running mean: -2.868259
episode 5247.000000, reward total was -1.000000. running mean: -2.849576
episode 5248.000000, reward total was 9.000000. running mean: -2.731080
episode 5249.000000, reward total was 5.000000. running mean: -2.653770
episode 5250.000000, reward total was -9.000000. running mean: -2.717232
episode 5251.000000, reward total was 3.000000. running mean: -2.660060
episode 5252.000000, reward total was 5.000000. running mean: -2.583459
episode 5253.000000, reward total was 3.000000. running mean: -2.527624
episode 5254.000000, reward total was 8.000000. running mean: -2.422348
episode 5255.000000, reward total was 9.000000. running mean: -2.308125
episode 5256.000000, reward total was 5.000000. running mean: -2.235043
episode 5257.000000, reward total was 6.000000. running mean: -2.152693
episode 5258.000000, reward total was 5.000000. running mean

episode 5358.000000, reward total was -1.000000. running mean: -2.132480
episode 5359.000000, reward total was 2.000000. running mean: -2.091156
episode 5360.000000, reward total was -1.000000. running mean: -2.080244
episode 5361.000000, reward total was -7.000000. running mean: -2.129442
episode 5362.000000, reward total was -4.000000. running mean: -2.148147
episode 5363.000000, reward total was 8.000000. running mean: -2.046666
episode 5364.000000, reward total was -1.000000. running mean: -2.036199
episode 5365.000000, reward total was -1.000000. running mean: -2.025837
episode 5366.000000, reward total was -11.000000. running mean: -2.115579
episode 5367.000000, reward total was -8.000000. running mean: -2.174423
episode 5368.000000, reward total was -5.000000. running mean: -2.202679
episode 5369.000000, reward total was 3.000000. running mean: -2.150652
episode 5370.000000, reward total was 3.000000. running mean: -2.099145
episode 5371.000000, reward total was -1.000000. runni

episode 5471.000000, reward total was -9.000000. running mean: -2.482775
episode 5472.000000, reward total was -10.000000. running mean: -2.557947
episode 5473.000000, reward total was -9.000000. running mean: -2.622368
episode 5474.000000, reward total was 5.000000. running mean: -2.546144
episode 5475.000000, reward total was -2.000000. running mean: -2.540683
episode 5476.000000, reward total was 5.000000. running mean: -2.465276
episode 5477.000000, reward total was -13.000000. running mean: -2.570623
episode 5478.000000, reward total was -8.000000. running mean: -2.624917
episode 5479.000000, reward total was 6.000000. running mean: -2.538668
episode 5480.000000, reward total was -5.000000. running mean: -2.563281
episode 5481.000000, reward total was -13.000000. running mean: -2.667648
episode 5482.000000, reward total was -1.000000. running mean: -2.650972
episode 5483.000000, reward total was -5.000000. running mean: -2.674462
episode 5484.000000, reward total was -9.000000. ru